In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sql-murder-mystery-database/sql-murder-mystery.db


In [3]:
# import some useful libraries
import sqlite3 as sql
import numpy as np
import pandas as np

In [4]:
# Connecting to the database
conn = sql.connect('../input/sql-murder-mystery-database/sql-murder-mystery.db')

In [5]:
# Creating a list of all tables from SQL query
query1 = "SELECT name FROM sqlite_master WHERE type = 'table'"
table_names = list(pd.read_sql_query(query1, conn).values.flatten())[:-1]
table_names

['crime_scene_report',
 'drivers_license',
 'person',
 'facebook_event_checkin',
 'interview',
 'get_fit_now_member',
 'get_fit_now_check_in',
 'income']

In [6]:
# Creating Dataframes from SQL query
for name in table_names:
    query = f"select * from {name}"
    globals()[f"{name}"]  = pd.read_sql(query, conn)
    print(f"Dataframe '{name}' created.")

Dataframe 'crime_scene_report' created.
Dataframe 'drivers_license' created.
Dataframe 'person' created.
Dataframe 'facebook_event_checkin' created.
Dataframe 'interview' created.
Dataframe 'get_fit_now_member' created.
Dataframe 'get_fit_now_check_in' created.
Dataframe 'income' created.


In [7]:
# START OF INVESTIGATION
# 1. Let's investigate the main table "Crime Scene Report"

crime_scene_report.head()

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City


In [8]:
# 2. I spotted, that this report includes all crimes. Let's filter it with our city name as 'SQL City' and type of crime as 'murder'

crime_scene_report[(crime_scene_report["city"]=="SQL City") & (crime_scene_report["type"]=="murder")]

,date,type,description,city
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City
1227,20180115,murder,Security footage shows that there were 2 witne...,SQL City


In [9]:
# 3. I don't see the while description, so let's see the full length of crime description

for i in range(len(crime_scene_report["description"][(crime_scene_report["city"]=="SQL City") & (crime_scene_report["type"]=="murder")])):
    print(f"Description #{i} is as follows: '{crime_scene_report['description'][(crime_scene_report['city']=='SQL City') & (crime_scene_report['type']=='murder')].iloc[i]}'.")

Description #0 is as follows: 'REDACTED REDACTED REDACTED'.
Description #1 is as follows: 'Someone killed the guard! He took an arrow to the knee!'.
Description #2 is as follows: 'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'.


In [10]:
# 4.1 Let's focus on the description #2, which says, that first witness lives at the last house on "Northwestern Dr" Let's find him!

witness_1 = person[(person['address_street_name']=='Northwestern Dr')&(person['address_number']== max(person['address_number'][person['address_street_name']=='Northwestern Dr']))]
witness_1

,id,name,license_id,address_number,address_street_name,ssn
499,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [11]:
# 4.2 Let's focus now on the second witness. The second witness, named Annabel, lives somewhere on "Franklin Ave". Let's find her!

witness_2 = person[(person['address_street_name']=='Franklin Ave')&(person['name'].str.contains("Annabel", case=False, na=False))]
witness_2

,id,name,license_id,address_number,address_street_name,ssn
665,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [12]:
# 5.1 Let's see what our witness_1 said on interview.

interview['transcript'][interview['person_id']==witness_1['id'].iloc[0]].iloc[0]

'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".'

In [13]:
# 5.2 Let's see what our witness_2 said on interview.

interview['transcript'][interview['person_id']==witness_2['id'].iloc[0]].iloc[0]

'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'

In [84]:
suspect_1 = get_fit_now_member[(get_fit_now_member['membership_status']=='gold')&(get_fit_now_member['id'].str.contains("48Z", case=False, na=False))]
suspect_1

,id,person_id,name,membership_start_date,membership_status
181,48Z7A,28819,Joe Germuska,20160305,gold
182,48Z55,67318,Jeremy Bowers,20160101,gold


In [87]:
suspect_car = drivers_license[drivers_license['plate_number'].str.contains('H42W')]
suspect_car

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
915,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
3529,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
6240,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [88]:
suspect_2 = get_fit_now_check_in[(get_fit_now_check_in['check_in_date'] == 20180109)]
suspect_2

,membership_id,check_in_date,check_in_time,check_out_time
113,X0643,20180109,957,1164
889,UK1F2,20180109,344,518
1329,XTE42,20180109,486,1124
1446,1AE2H,20180109,461,944
1703,6LSTG,20180109,399,515
2137,7MWHJ,20180109,273,885
2435,GE5Q8,20180109,367,959
2700,48Z7A,20180109,1600,1730
2701,48Z55,20180109,1530,1700
2702,90081,20180109,1600,1700


In [136]:
# Suspect = 48Z7A
print(interview['transcript'][interview['person_id'] == suspect_1['person_id'].iloc[0]])

# Suspect = 48Z55 -- 67318
print(interview['transcript'][interview['person_id'] == suspect_1['person_id'].iloc[1]].iloc[0])

Series([], Name: transcript, dtype: object)
I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.



In [190]:
# If we filter drivers just by a car details, then we have a list of potential suspects in drivers_by_car
drivers_by_car = drivers_license[(drivers_license['car_make'] == 'Tesla') &
                (drivers_license['car_model'] == 'Model S')]


drivers_by_car

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
1105,202298,68,66,green,red,female,500123,Tesla,Model S
2054,291182,65,66,blue,red,female,08CM64,Tesla,Model S
4012,467669,70,65,green,white,male,W528W7,Tesla,Model S
5975,638311,72,63,amber,white,male,0M846T,Tesla,Model S
6428,682231,51,80,green,grey,female,3Z0M62,Tesla,Model S
7083,736081,79,69,brown,red,male,GCAQ6Y,Tesla,Model S
9078,918773,48,65,black,red,female,917UU3,Tesla,Model S


In [191]:
# If we filter drivers just by all details, then we have a list of potential suspects in drivers_by_all
drivers_by_all = drivers_license[(drivers_license['gender'] == 'female') &
                (drivers_license['hair_color'] == 'red') &
                (drivers_license['car_make'] == 'Tesla') &
                (drivers_license['car_model'] == 'Model S')]

drivers_by_all

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
1105,202298,68,66,green,red,female,500123,Tesla,Model S
2054,291182,65,66,blue,red,female,08CM64,Tesla,Model S
9078,918773,48,65,black,red,female,917UU3,Tesla,Model S


In [164]:
concert_spectators = facebook_event_checkin[(facebook_event_checkin['event_name'] == 'SQL Symphony Concert') & 
                       (facebook_event_checkin['date'] > 20171200) &
                       (facebook_event_checkin['date'] < 20171232)]

attendance = concert_spectators.groupby('person_id')['person_id'].count()

ids_with_count_3 = attendance[attendance == 3].index.tolist()
ids_with_count_3

[24556, 99716]

In [197]:
# for id in range(len(ids_with_count_3)):
#     print(drivers_license[(drivers_license['id'] == ids_with_count_3[id])])
#     # no connections

# person[person['id'] == ids_with_count_3[0]]
person[person['id'] == ids_with_count_3[1]]

,id,name,license_id,address_number,address_street_name,ssn
9985,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [207]:
# Suspect = 99716
# print(interview['transcript'][interview['person_id'] == 99716])
person[person['id'] == 99716]
income[income['ssn'] == 987756388]
drivers_license[drivers_license['id'] == 202298]

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
1105,202298,68,66,green,red,female,500123,Tesla,Model S


In [158]:
# Dataframe 'crime_scene_report' created.
# Dataframe 'drivers_license' created.
# Dataframe 'person' created.
# Dataframe 'facebook_event_checkin' created.
# Dataframe 'interview' created.
# Dataframe 'get_fit_now_member' created.
# Dataframe 'get_fit_now_check_in' created.
# Dataframe 'income' created.